In [94]:
# 검색어로 크롬 실행, 실행결과의 링크 추출, 링크를 타고 각 페이지 접속, 페이지 내용 전체 크롤링, text 추출하고, 단어 리스트로 만드는 코드

import re
import requests
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")


In [95]:
def google_search_result(input_word):
    baseUrl = 'https://www.google.com/search?q='

    #plusUrl = input('무엇을 검색할까요? :')
    plusUrl = input_word

    # url = baseUrl + quote_plus(plusUrl)
    url = baseUrl + plusUrl
    # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_mac_ver_90', chrome_options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, features="html.parser")

    v = soup.select('.yuRUbf')

    search_title_result = []
    search_linked_contents_result = []
    for i in v:
        #print(i.select_one('.LC20lb.DKV0Md').text)
        search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
        #print(i.a.attrs['href'])
        search_linked_contents_result.append(i.a.attrs['href'])
        #print()


    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, 단어만 추리고, 다시 주요키워드를 추출한다.
    
    get_all_linked_web_data = []
    for linked_page in search_linked_contents_result:
        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    return item_extract



In [96]:
get_result = google_search_result('bitcoin') 

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [100]:
get_result_str = str(get_result)
get_result_str

'                                                                                                                                                                                                                                                                                      Education                    General         Dictionary         Economics         Corporate Finance         Roth IRA         Stocks         Mutual Funds         ETFs         401 k              Investing Trading         Investing Essentials         Fundamental Analysis         Portfolio Management         Trading Essentials         Technical Analysis         Risk Management                     Markets                    News         Company News         Markets News         Trading News         Political News         Trends             Popular Stocks         Apple  AAPL          Tesla  TSLA          Amazon  AMZN          AMD  AMD          Facebook  FB          Netflix  NFLX                      Simulator         

In [106]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [107]:
result_cln = cleaning_data(get_result_str)
result_cln

['facilitates',
 'confused',
 'magazine',
 'stated',
 'standard',
 'wallet',
 'since',
 'involving',
 'scale',
 'released',
 'iu',
 'applies',
 'adam',
 'cryptocurrency',
 'participating',
 'feb',
 'gains',
 'choose',
 'guide',
 'burstcoin',
 'operate',
 'larger',
 'later',
 'consumer',
 'alone',
 'venture',
 'warranties',
 'error',
 'cash',
 'video',
 'primary',
 'widespread',
 'verify',
 'trader',
 'jan',
 'u',
 'management',
 'engaged',
 'volume',
 'touchstart',
 'preload',
 '1997',
 'dealing',
 'defaultpositionx',
 'lot',
 'defaultpositiony',
 'supporter',
 'currency',
 'processing',
 'inherent',
 'price',
 'world',
 'bitwage',
 'world',
 'department',
 'operational',
 'pretty',
 'client',
 'formatadsizearray',
 'trics',
 'averse',
 'site',
 'created',
 'cons',
 'hashcash',
 'unlike',
 'terms',
 'total',
 'carry',
 'illegal',
 'industry',
 'something',
 'new',
 'unlikely',
 'declining',
 'affiliatelinkrewriter',
 'cold',
 'payout',
 'published',
 'finney’s',
 'motivate',
 'amazon',